<a href="https://colab.research.google.com/github/Rajora0/nlp_studies/blob/main/Classifica%C3%A7%C3%A3o_multilabel_de_textos_m%C3%BAltiplos_contextos_em_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Classificação multilabel de textos: múltiplos contextos em NLP

In [ ]:
!pip install scikit-learn==0.24.1 -qq
!pip install scikit-multilearn==0.2.0 -qq

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yellowbrick 1.4 requires scikit-learn>=1.0.0, but you have scikit-learn 0.24.1 which is incompatible.


## Montar Drive

In [ ]:
from google.colab import drive
drive.mount('/gdrive/')

!ls '/gdrive/MyDrive/NLP_ESTUDOS/Dados/'

Drive already mounted at /gdrive/; to attempt to forcibly remount, call drive.mount("/gdrive/", force_remount=True).
archive.zip	     imdb-reviews-pt-br.csv	 stackoverflow_perguntas.csv
corretor-master      stackoverflow_espanhol.csv  stackoverflow_portugues.csv
corretor-master.zip  stackoverflow_ingles.csv	 word2vec


## ETL

In [ ]:
#!wget https://raw.githubusercontent.com/alura-cursos/alura_classificacao_multilabel/master/dataset/stackoverflow_perguntas.csv -O /gdrive/MyDrive/NLP_ESTUDOS/Dados/stackoverflow_perguntas.csv

## Carregando os dados

In [ ]:
import numpy as np
import pandas as pd

# import spacy
# import warnings
# warnings.filterwarnings("ignore")

In [ ]:
path = '/gdrive/MyDrive/NLP_ESTUDOS/Dados/'

In [ ]:
perguntas = pd.read_csv(path+'stackoverflow_perguntas.csv')
perguntas.sample(n=10)

,Perguntas,Tags
3589,Tenho um motor de busca simples que faz a busc...,html
915,Este código remove todas as divs menos selecio...,jquery html
2872,Preciso fazer o processamento assíncrono de sé...,angular
3917,Estou utilizando o código abaixo para pré visu...,html
1764,Meu código é esse: CODE Gostaria de saber s...,jquery
4157,"Estou interessado nos sistemas de live ex, fac...",html angular node.js
4874,Eu tento criar um teste com várias perguntas e...,angular
228,Segue o que fiz inicialmente: CODE ... COD...,node.js
5051,Utilizo um service para fazer com que cada rot...,angular
3430,"Gostaria de enviar os valores do checkbox, som...",html


In [ ]:
perguntas.Perguntas.shape[0]

5408

In [ ]:
perguntas.Tags.nunique()

37

In [ ]:
perguntas.Tags.unique()

array(['node.js', 'jquery', 'html', 'html angular ', 'html ', 'angular',
       'angular ', 'jquery html  ', 'jquery ', 'jquery html',
       'jquery html ', 'html angular', 'angular node.js ', 'html  ',
       'jquery html angular', 'node.js ', 'html jquery', 'html jquery ',
       'jquery angular  ', 'html node.js', 'jquery  ', 'angular node.js',
       'jquery angular', 'html node.js ', 'jquery node.js ', 'angular  ',
       'jquery angular ', 'jquery html angular ', 'node.js html ',
       ' node.js', 'node.js html', 'html angular  ', 'jquery node.js',
       'angular html', 'html angular  node.js', 'jquery html node.js',
       'html angular node.js'], dtype=object)

In [ ]:
lista_de_tags = list()

for tags in perguntas.Tags.unique():
  for tag in tags.split():
    if tag not in lista_de_tags:
      lista_de_tags.append(tag)

lista_de_tags

['node.js', 'jquery', 'html', 'angular']

## Get Dummies Multilabel

In [ ]:
def nova_coluna(lista_tags, dataframe, nome_tags):
    for tag in lista_tags:
        coluna = list()
        for linha_tag in dataframe[nome_tags]:
            if tag in linha_tag:
                coluna.append(1)
            else:
                coluna.append(0)
        dataframe[tag] = coluna
        
nova_coluna(lista_de_tags, perguntas, "Tags")
perguntas.head()

,Perguntas,Tags,node.js,jquery,html,angular
0,Possuo um projeto Node.js porém preciso criar ...,node.js,1,0,0,0
1,"Gostaria de fazer testes unitários no Node.js,...",node.js,1,0,0,0
2,Como inverter a ordem com que o jQuery itera u...,jquery,0,1,0,0
3,Eu tenho uma página onde pretendo utilizar um ...,html,0,0,1,0
4,Como exibir os dados retornados do FireStore e...,html angular,0,0,1,1


## Zip para classificação multilabel


In [ ]:
lista_zip_tags = list(zip(perguntas[lista_de_tags[0]],
                     perguntas[lista_de_tags[1]],
                     perguntas[lista_de_tags[2]],
                     perguntas[lista_de_tags[3]]))


In [ ]:
perguntas['todas_tags'] = lista_zip_tags
perguntas.head()

,Perguntas,Tags,node.js,jquery,html,angular,todas_tags
0,Possuo um projeto Node.js porém preciso criar ...,node.js,1,0,0,0,"(1, 0, 0, 0)"
1,"Gostaria de fazer testes unitários no Node.js,...",node.js,1,0,0,0,"(1, 0, 0, 0)"
2,Como inverter a ordem com que o jQuery itera u...,jquery,0,1,0,0,"(0, 1, 0, 0)"
3,Eu tenho uma página onde pretendo utilizar um ...,html,0,0,1,0,"(0, 0, 1, 0)"
4,Como exibir os dados retornados do FireStore e...,html angular,0,0,1,1,"(0, 0, 1, 1)"


## Dados de treino e teste

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
perguntas_train, perguntas_test, tag_train, tag_test = train_test_split(
    perguntas.Perguntas,
    perguntas.todas_tags,
    test_size = 0.2,
    random_state = 123
)

In [ ]:
perguntas_train

1577       array1 = [1,2,3];   array2 = ["um","dois","...
1927    Não sei se fui claro no título, mas quem é da ...
3409    Alguém sabe me dizer qual a melhor forma de re...
4606    Estou com problemas ao tentar validar campos d...
5237    Preciso copiar um valor de dentro de um CODE  ...
                              ...                        
5218    Tenho um sisteminha, para mudar o layout da pá...
4060    Como fazer alto scoll ao carregar a página?  E...
1346    Explicação:  Tenho uma CODE  pai que contém du...
3454    Estou querendo fazer um sistema onde eu iria t...
3582    Galera eu to com um problemão, ja pesquisei ba...
Name: Perguntas, Length: 4326, dtype: object

In [ ]:
print(tag_train)
print(type(tag_train))

1577    (0, 1, 0, 0)
1927    (0, 1, 0, 0)
3409    (0, 0, 1, 0)
4606    (0, 0, 1, 0)
5237    (0, 1, 0, 0)
            ...     
5218    (0, 0, 1, 0)
4060    (0, 1, 0, 0)
1346    (0, 1, 1, 0)
3454    (0, 0, 1, 0)
3582    (0, 1, 1, 0)
Name: todas_tags, Length: 4326, dtype: object
<class 'pandas.core.series.Series'>


In [ ]:
list(map(lambda x: len(x), [perguntas_train, perguntas_test, tag_train, tag_test]))

[4326, 1082, 4326, 1082]

## TF-IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
vectorizer = TfidfVectorizer(max_features=5000,max_df=0.85)
vectorizer

TfidfVectorizer(max_df=0.85, max_features=5000)

In [ ]:
vectorizer.fit(perguntas.Perguntas)

TfidfVectorizer(max_df=0.85, max_features=5000)

In [ ]:
perguntas_train_tfid = vectorizer.transform(perguntas_train)
perguntas_test_tfid = vectorizer.transform(perguntas_test)

In [ ]:
tags_treino_array = np.asarray(list(tag_train))
tags_test_array = np.asarray(list(tag_test))

In [ ]:
tags_treino_array

array([[0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 0, 1, 0],
       ...,
       [0, 1, 1, 0],
       [0, 0, 1, 0],
       [0, 1, 1, 0]])

## OneVsRest


In [ ]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression

RL = LogisticRegression(solver = 'lbfgs')
OR = OneVsRestClassifier(RL)
OR.fit(perguntas_train_tfid,tags_treino_array)

OneVsRestClassifier(estimator=LogisticRegression())

In [ ]:
results = OR.score(perguntas_test_tfid,tags_test_array)
print(round(results*100,2))

41.68


In [ ]:
OR.predict(perguntas_test_tfid)

array([[0, 1, 0, 0],
       [0, 0, 1, 0],
       [1, 0, 0, 0],
       ...,
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 0, 0, 0]])

In [ ]:
tags_test_array

array([[0, 1, 0, 0],
       [0, 0, 1, 0],
       [1, 0, 0, 0],
       ...,
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 0, 0, 1]])

In [ ]:
perguntas.todas_tags.nunique()

13

## Metrica Hamming Loss

In [ ]:
from sklearn.metrics import hamming_loss

In [ ]:
previsão_or = OR.predict(perguntas_test_tfid)
hamming_loss_or = hamming_loss(tags_test_array,previsão_or)
print(round(hamming_loss_or,2))

0.19


In [ ]:
perguntas.corr()

,node.js,jquery,html,angular
node.js,1.000000,-0.321485,-0.273523,-0.101787
jquery,-0.321485,1.000000,-0.253977,-0.366269
html,-0.273523,-0.253977,1.000000,-0.286706
angular,-0.101787,-0.366269,-0.286706,1.000000


## Classificação em cadeia

In [ ]:
from skmultilearn.problem_transform import ClassifierChain

In [ ]:
RL = LogisticRegression(solver = 'lbfgs')
CC = ClassifierChain(RL)
CC.fit(perguntas_train_tfid,tags_treino_array)

ClassifierChain(classifier=LogisticRegression(), require_dense=[True, True])

In [ ]:
results = CC.score(perguntas_test_tfid,tags_test_array)
print(round(results*100,2))

49.82


In [ ]:
previsão_cc = CC.predict(perguntas_test_tfid)
hamming_loss_cc = hamming_loss(tags_test_array,previsão_cc)
print(round(hamming_loss_cc,2))

0.21


## Relevância Binaria com skmultilearn

In [ ]:
from skmultilearn.problem_transform import BinaryRelevance

In [ ]:
RL = LogisticRegression(solver = 'lbfgs')
BR = BinaryRelevance(RL)
BR.fit(perguntas_train_tfid,tags_treino_array)

results = BR.score(perguntas_test_tfid,tags_test_array)
print(round(results*100,2))

previsão_br = BR.predict(perguntas_test_tfid)
hamming_loss_br = hamming_loss(tags_test_array,previsão_br)
print(round(hamming_loss_br,2))

41.68
0.19


## ML-KNN

In [ ]:
from skmultilearn.adapt import MLkNN
from sklearn.neighbors import NearestNeighbors


In [ ]:
CKNN = MLkNN(k=10)
CKNN.fit(perguntas_train_tfid,tags_treino_array)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:72: FutureWarning: Pass n_neighbors=10 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


MLkNN()

In [ ]:
results = CKNN.score(perguntas_test_tfid,tags_test_array)
print(round(results*100,2))

previsão_knn= CKNN.predict(perguntas_test_tfid)
hamming_loss_knn = hamming_loss(tags_test_array,previsão_knn)
print(round(hamming_loss_knn,2))

32.53
0.25


## Resultado final

In [ ]:
result = {'perguntas':perguntas_test.values,
          'tags_real':list(tag_test),
          'BR':list(previsão_br.toarray()),
          'CC':list(previsão_cc.toarray()),
          'CKNN':list(previsão_knn.toarray())}

In [ ]:
pd.DataFrame(result)

,perguntas,tags_real,BR,CC,CKNN
0,estou com conflito entre o CODE e os CODE ...,"(0, 1, 0, 0)","[0, 1, 0, 0]","[0.0, 1.0, 0.0, 0.0]","[0, 0, 0, 0]"
1,Estou fazendo um site que eu sou obrigado a us...,"(0, 0, 1, 0)","[0, 0, 1, 0]","[0.0, 0.0, 1.0, 0.0]","[0, 1, 1, 0]"
2,Recentemente fiz um refactor do meu código par...,"(1, 0, 0, 0)","[1, 0, 0, 0]","[1.0, 0.0, 0.0, 0.0]","[1, 0, 0, 0]"
3,Eu tenho esse código em CODE que passo valore...,"(0, 1, 1, 0)","[0, 1, 0, 0]","[0.0, 1.0, 0.0, 0.0]","[0, 1, 1, 0]"
4,"Olá, em minha função tem o evento CODE que de...","(0, 1, 1, 0)","[0, 1, 0, 0]","[0.0, 1.0, 0.0, 0.0]","[0, 1, 1, 0]"
...,...,...,...,...,...
1077,Estou a desenvolver um website em jQuery. E at...,"(0, 1, 0, 0)","[0, 1, 1, 0]","[0.0, 1.0, 1.0, 0.0]","[0, 0, 1, 0]"
1078,Estou usando este plugin - jquery autocomplete...,"(0, 1, 0, 0)","[0, 1, 0, 0]","[0.0, 1.0, 0.0, 0.0]","[0, 1, 0, 0]"
1079,"Tenho o seguinte jQuery: CODE Nisto, quanti...","(0, 1, 0, 0)","[0, 1, 0, 0]","[0.0, 1.0, 0.0, 0.0]","[0, 1, 0, 0]"
1080,Estou usando o SimpleModal Contact Form de Eri...,"(0, 1, 0, 0)","[0, 1, 0, 0]","[0.0, 1.0, 0.0, 0.0]","[0, 1, 0, 0]"
